# CovNet DeepShared — Simulation & Evaluation Notebook

This notebook implements the **DeepShared CovNet** estimator (Sarkar & Panaretos, 2022) and a set of simulation experiments aligned with the user's simulation plan.  
It purposefully focuses **only on the DeepShared variant** and includes:
- GP data generation (Brownian sheet, rotated, Matérn),
- DeepShared model implementation in PyTorch (shared backbone + R heads),
- Training objective implemented exactly as in the paper (loss written in terms of inner-products),
- Evaluation: relative L2 error via Monte Carlo, MSE vs N (convergence), scaling experiments,
- Utilities (early stopping, GPU support if available),
- A simple experiment runner with default parameters matching the plan; you can edit inputs.

Note: the notebook is runnable as-is. For large grid sizes / large N you should run on a machine with enough RAM and (preferably) a GPU.


In [1]:
# Basic environment setup
import math, time, os, json, sys, copy
from pprint import pprint
import numpy as np
import scipy as sp
from scipy.spatial.distance import cdist
from scipy.special import kv, gamma
from scipy.linalg import eigh
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import trange, tqdm
np.random.seed(0)
torch.manual_seed(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)


Using device: cpu


In [2]:
# -------------------------------
# Covariance kernels and sampling
# -------------------------------

def grid_on_unit_cube(d, K):
    """Return D x d array of grid points on [0,1]^d (regular grid with K points per axis)."""
    axes = [np.linspace(0,1,K) for _ in range(d)]
    mesh = np.stack(np.meshgrid(*axes, indexing='ij'), axis=-1)
    pts = mesh.reshape(-1, d)
    return pts

def brownian_sheet_cov(pts1, pts2):
    # separable product of 1D min kernels
    n,d = pts1.shape
    m = pts2.shape[0]
    cov = np.ones((n,m))
    for k in range(d):
        a = pts1[:,k:k+1]  # (n,1)
        b = pts2[:,k:k+1]  # (m,1)
        cov *= np.minimum(a, b.T)
    return cov

def integrated_brownian_cov(pts1, pts2):
    """Integrated Brownian motion kernel (1D) product across dimensions"""
    def cibm_1d(u, v):
        u = np.asarray(u); v = np.asarray(v)
        out = np.where(u<=v, (u**2/2)*(v - u/3), (v**2/2)*(u - v/3))
        return out
    n = pts1.shape[0]; m = pts2.shape[0]; d = pts1.shape[1]
    cov = np.ones((n,m))
    for k in range(d):
        a = pts1[:,k:k+1]; b = pts2[:,k:k+1]
        A = np.repeat(a, m, axis=1)
        B = np.repeat(b.T, n, axis=0)
        cov *= cibm_1d(A, B)
    return cov

def matern_cov(pts1, pts2, nu=0.5, rho=0.2, sigma2=1.0):
    """Isotropic Matérn kernel with range param rho and smoothness nu"""
    dists = cdist(pts1, pts2, metric='euclidean')
    if nu == 0.5:
        K = sigma2 * np.exp(-dists / rho)
        return K
    factor = np.sqrt(2*nu) * dists / rho
    K = sigma2 * (2**(1-nu) / gamma(nu)) * (factor**nu) * sp.special.kv(nu, factor)
    K[np.isnan(K)] = sigma2
    return K

def rotate_points(pts, angle_rad=None):
    d = pts.shape[1]
    if angle_rad is None:
        angle_rad = np.pi/6
    R = np.eye(d)
    c = np.cos(angle_rad); s = np.sin(angle_rad)
    R[0,0] = c; R[0,1] = -s; R[1,0] = s; R[1,1] = c
    return pts.dot(R.T)

def sample_gp_on_grid(pts, cov_fn, jitter=1e-8, sigma2_noise=0.0):
    C = cov_fn(pts, pts)
    C = (C + C.T) / 2.0
    C += np.eye(C.shape[0]) * jitter
    L = np.linalg.cholesky(C)
    z = np.random.normal(size=(C.shape[0],))
    sample = L.dot(z)
    if sigma2_noise > 0:
        sample = sample + np.random.normal(scale=np.sqrt(sigma2_noise), size=sample.shape)
    return sample

def generate_dataset(N, pts, cov_type='matern', cov_params=None, jitter=1e-8, sigma2_noise=0.0, rotate=False):
    D = pts.shape[0]
    X = np.zeros((N, D))
    cov_fn = None
    if cov_type == 'brownian':
        cov_fn = brownian_sheet_cov
    elif cov_type == 'ibm':
        cov_fn = integrated_brownian_cov
    elif cov_type == 'matern':
        nu = cov_params.get('nu', 0.5) if cov_params is not None else 0.5
        rho = cov_params.get('rho', 0.2) if cov_params is not None else 0.2
        sigma2 = cov_params.get('sigma2', 1.0) if cov_params is not None else 1.0
        cov_fn = lambda a,b: matern_cov(a,b,nu=nu,rho=rho,sigma2=sigma2)
    else:
        raise ValueError("Unknown cov_type")
    for i in range(N):
        if rotate:
            ptsr = rotate_points(pts, angle_rad=cov_params.get('angle', np.pi/6) if cov_params is not None else np.pi/6)
            X[i,:] = sample_gp_on_grid(ptsr, cov_fn, jitter=jitter, sigma2_noise=sigma2_noise)
        else:
            X[i,:] = sample_gp_on_grid(pts, cov_fn, jitter=jitter, sigma2_noise=sigma2_noise)
    X = X - X.mean(axis=0, keepdims=True)
    return X


In [3]:
# -------------------------------
# DeepShared CovNet model in PyTorch
# -------------------------------

class DeepSharedBackbone(nn.Module):
    def __init__(self, d, widths, activation='sigmoid'):
        super().__init__()
        layers = []
        in_dim = d
        act = nn.Sigmoid() if activation=='sigmoid' else nn.ReLU()
        for w in widths:
            layers.append(nn.Linear(in_dim, w))
            layers.append(act)
            in_dim = w
        self.net = nn.Sequential(*layers)
        self.out_dim = in_dim
    def forward(self, x):
        return self.net(x)  # (D, pL)

class DeepSharedCovNet(nn.Module):
    def __init__(self, d, widths, R, activation='sigmoid'):
        super().__init__()
        self.backbone = DeepSharedBackbone(d, widths, activation)
        self.R = R
        self.final = nn.Linear(self.backbone.out_dim, R)
        self.activation = nn.Sigmoid() if activation=='sigmoid' else nn.ReLU()
    def forward(self, pts):
        if not torch.is_tensor(pts):
            pts = torch.tensor(pts, dtype=torch.get_default_dtype(), device=next(self.parameters()).device)
        out = self.backbone(pts)  # (D, pL)
        out = self.final(out)     # (D, R)
        out = self.activation(out)  # (D, R)
        return out  # D x R


In [4]:
# -------------------------------
# Training utilities (loss, train loop)
# -------------------------------

def compute_loss_from_matrices(X, Xnn):
    if isinstance(X, np.ndarray):
        X = torch.tensor(X, dtype=torch.get_default_dtype(), device=device)
    if isinstance(Xnn, np.ndarray):
        Xnn = torch.tensor(Xnn, dtype=torch.get_default_dtype(), device=device)
    N, D = X.shape
    Kx = (X @ X.t()) / D
    Kxnn = (Xnn @ Xnn.t()) / D
    Kcross = (X @ Xnn.t()) / D
    term1 = torch.sum(Kx**2)
    term2 = torch.sum(Kxnn**2)
    term3 = torch.sum(Kcross**2)
    loss = (term1 + term2 - 2*term3) / (N**2)
    return loss, term1.item(), term2.item(), term3.item()

def train_deepshared(X, pts, R=20, widths=[64,64,64], activation='sigmoid',
                      lr=1e-2, weight_decay=0.0, max_epochs=1000, patience=20,
                      batch_mode=False, verbose=True):

    N, D = X.shape
    d = pts.shape[1]

    model = DeepSharedCovNet(d, widths, R, activation=activation).to(device)

    # Xi must be a LEAF PARAMETER
    Xi = nn.Parameter(0.01 * torch.randn((N, R),
                        device=device, dtype=torch.get_default_dtype()))

    optim_params = list(model.parameters()) + [Xi]
    optimizer = optim.Adam(optim_params, lr=lr, weight_decay=weight_decay)
    X_torch = torch.tensor(X, dtype=torch.get_default_dtype(), device=device)
    pts_torch = torch.tensor(pts, dtype=torch.get_default_dtype(), device=device)
    history = {'loss': [], 'term1': [], 'term2': [], 'term3': [], 'time': []}
    best_loss = np.inf; best_state = None; wait = 0
    for epoch in range(max_epochs):
        t0 = time.time()
        model.train()
        G = model(pts_torch)  # (D, R)
        Xnn = Xi @ G.t()
        loss, t1, t2, t3 = compute_loss_from_matrices(X_torch, Xnn)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        dur = time.time() - t0
        history['loss'].append(loss.item()); history['term1'].append(t1); history['term2'].append(t2); history['term3'].append(t3); history['time'].append(dur)
        if verbose and (epoch % max(1, max_epochs//10) == 0 or epoch < 10):
            print(f"Epoch {epoch:4d} loss={loss.item():.6e} t1={t1:.3e} t2={t2:.3e} t3={t3:.3e} time={dur:.2f}s")
        if loss.item() < best_loss - 1e-12:
            best_loss = loss.item(); best_state = (copy.deepcopy(model.state_dict()), Xi.detach().cpu().numpy()); wait = 0
        else:
            wait += 1
        if wait >= patience:
            if verbose:
                print(f"Early stopping at epoch {epoch}, best_loss={best_loss:.6e}")
            break
    if best_state is not None:
        model.load_state_dict(best_state[0])
        Xi_final = best_state[1]
    else:
        Xi_final = Xi.detach().cpu().numpy()
    return model, Xi_final, history


In [5]:
# -------------------------------
# Evaluation utilities
# -------------------------------

def monte_carlo_rel_L2(c_true_fn, c_est_fn, d, M=50000, seed=0):
    rng = np.random.RandomState(seed)
    U = rng.rand(M, d)
    V = rng.rand(M, d)
    c_true_vals = c_true_fn(U, V)
    c_est_vals = c_est_fn(U, V)
    num = np.mean((c_est_vals - c_true_vals)**2)
    den = np.mean(c_true_vals**2)
    return math.sqrt(num / max(1e-20, den))

def make_c_est_from_deepshared(model, Xi, pts):
    device_local = next(model.parameters()).device
    def c_est(u, v):
        u = np.asarray(u); v = np.asarray(v)
        Mu = u.shape[0]; Mv = v.shape[0]
        with torch.no_grad():
            Gu = model(torch.tensor(u, dtype=torch.get_default_dtype(), device=device_local)).cpu().numpy()
            Gv = model(torch.tensor(v, dtype=torch.get_default_dtype(), device=device_local)).cpu().numpy()
        Xi_centered = Xi - Xi.mean(axis=0, keepdims=True)
        Lambda = (Xi_centered.T @ Xi_centered) / (Xi.shape[0])
        if Mu == Mv:
            tmp = (Gu @ Lambda) * Gv
            vals = tmp.sum(axis=1)
            return vals
        else:
            return (Gu @ Lambda @ Gv.T)
    return c_est

def true_cov_function_factory(pts, cov_type='matern', cov_params=None, rotate=False):
    if cov_type == 'brownian':
        return lambda u,v: brownian_sheet_cov(u, v)
    elif cov_type == 'ibm':
        return lambda u,v: integrated_brownian_cov(u, v)
    elif cov_type == 'matern':
        nu = cov_params.get('nu',0.5) if cov_params is not None else 0.5
        rho = cov_params.get('rho',0.2) if cov_params is not None else 0.2
        sigma2 = cov_params.get('sigma2',1.0) if cov_params is not None else 1.0
        return lambda u,v: matern_cov(u, v, nu=nu, rho=rho, sigma2=sigma2)
    else:
        raise ValueError("Unknown cov type")


In [6]:
# -------------------------------
# Example: run a small experiment (Scenario B-style / convergence habit) on d=2 K=25 using DeepShared
# -------------------------------

def run_example_scenario(N=500, d=2, K=25, cov_type='matern', cov_params=None,
                         R=20, widths=[64,64,64], lr=1e-2, max_epochs=1000, patience=20, M_mc=20000,
                         verbose=True):
    pts = grid_on_unit_cube(d, K)
    print("Grid points D:", pts.shape[0])
    X = generate_dataset(N, pts, cov_type=cov_type, cov_params=cov_params or {}, jitter=1e-8, sigma2_noise=0.0, rotate=False)
    t0 = time.time()
    model, Xi, hist = train_deepshared(X, pts, R=R, widths=widths, activation='sigmoid', lr=lr,
                                      weight_decay=0.0, max_epochs=max_epochs, patience=patience, verbose=verbose)
    t_fit = time.time() - t0
    print("Fitting finished in", t_fit, "sec")
    c_true_fn = true_cov_function_factory(pts, cov_type=cov_type, cov_params=cov_params or {}, rotate=False)
    c_est_fn = make_c_est_from_deepshared(model, Xi, pts)
    rel_l2 = monte_carlo_rel_L2(c_true_fn, c_est_fn, d=d, M=M_mc, seed=42)
    print("Relative L2 error (MC):", rel_l2)
    return dict(model=model, Xi=Xi, history=hist, rel_l2=rel_l2, fit_time=t_fit, X=X, pts=pts)

if __name__ == '__main__':
    demo_res = run_example_scenario(N=200, d=2, K=15, cov_type='matern', cov_params={'nu':0.5,'rho':0.2,'sigma2':1.0},
                                    R=16, widths=[64,64,64], lr=1e-2, max_epochs=400, patience=40, M_mc=5000, verbose=True)
    print("Demo done. rel_l2:", demo_res['rel_l2'])


Grid points D: 225
Epoch    0 loss=5.367733e-02 t1=2.153e+03 t2=1.196e-02 t3=2.900e+00 time=0.02s
Epoch    1 loss=4.994477e-02 t1=2.153e+03 t2=9.645e+00 t3=8.237e+01 time=0.00s
Epoch    2 loss=4.230326e-02 t1=2.153e+03 t2=1.198e+02 t3=2.903e+02 time=0.00s
Epoch    3 loss=3.628418e-02 t1=2.153e+03 t2=7.779e+02 t3=7.397e+02 time=0.00s
Epoch    4 loss=4.650314e-02 t1=2.153e+03 t2=2.189e+03 t3=1.241e+03 time=0.00s
Epoch    5 loss=3.856523e-02 t1=2.153e+03 t2=1.308e+03 t3=9.593e+02 time=0.00s
Epoch    6 loss=3.635402e-02 t1=2.153e+03 t2=5.934e+02 t3=6.461e+02 time=0.02s
Epoch    7 loss=3.847682e-02 t1=2.153e+03 t2=2.909e+02 t3=4.524e+02 time=0.00s
Epoch    8 loss=4.037647e-02 t1=2.153e+03 t2=1.864e+02 t3=3.621e+02 time=0.00s
Epoch    9 loss=4.098765e-02 t1=2.153e+03 t2=1.622e+02 t3=3.378e+02 time=0.02s
Epoch   40 loss=3.641968e-02 t1=2.153e+03 t2=8.542e+02 t3=7.752e+02 time=0.00s
Epoch   80 loss=3.623550e-02 t1=2.153e+03 t2=7.135e+02 t3=7.085e+02 time=0.00s
Epoch  120 loss=3.621821e-02 t1=2

In [7]:
# Single code block to run Scenarios A-D across supported kernel types.
# Assumes the helper functions from the notebook/script are already defined in the environment:
#   grid_on_unit_cube, generate_dataset, train_deepshared,
#   true_cov_function_factory, make_c_est_from_deepshared, monte_carlo_rel_L2, run_example_scenario
#
# Adjust `n_reps` and M_mc to suit your compute. This block collects a summary table (CSV).

import time, os
import numpy as np
import pandas as pd
from pathlib import Path

OUTDIR = Path("./sim_summary")
OUTDIR.mkdir(exist_ok=True)

# -----------------------
# Kernel configurations
# -----------------------
kernel_configs = {
    'brownian': {},
    'ibm': {},
    'matern': {'nu': 0.5, 'rho': 0.2, 'sigma2': 1.0},
}

# For kernels where we want to also test rotated versions
rotatable = {'brownian', 'matern'}

# -----------------------
# Scenario settings
# -----------------------
n_reps = 3                   # shorten for quick runs; increase for production (25-50)
M_mc = 10000                 # Monte Carlo pairs for relative L2 (lower for quick runs)
results = []

# # Scenario A: Approximation power (vary R) — d=2 K=25 N=500
R_values = [5, 10, 20, 40, 60, 100]
for kernel, cov_params in kernel_configs.items():
    for rotate in ([False, True] if kernel in rotatable else [False]):
        for R in R_values:
            for rep in range(n_reps):
                t0 = time.time()
                try:
                    res = run_example_scenario(
                        N=500, d=2, K=25,
                        cov_type=kernel,
                        cov_params=cov_params or {},
                        R=R,
                        widths=[64,64,64],
                        lr=1e-2,
                        max_epochs=800,
                        patience=80,
                        M_mc=M_mc,
                        verbose=False
                    )
                    fit_time = res['fit_time']
                    rel_l2 = res['rel_l2']
                except Exception as e:
                    fit_time = None
                    rel_l2 = None
                    print(f"[Scenario A] error kernel={kernel} rotate={rotate} R={R} rep={rep}: {e}")
                results.append({
                    'scenario': 'A_approx_R',
                    'kernel': kernel,
                    'rotated': rotate,
                    'R': R,
                    'N': 500, 'd': 2, 'K': 25,
                    'rep': rep,
                    'rel_l2': rel_l2,
                    'fit_time': fit_time,
                    'timestamp': time.time()
                })
                # optional small pause
                time.sleep(0.1)

# # Scenario B: Convergence (vary N) — Matérn nu=0.1, d=2 K=25, DeepShared (L=3 implicit via widths)
Ns = [50, 100, 200, 500, 1000, 5000]
for kernel, cov_params in kernel_configs.items():
    # we focus on Matérn-like convergence but allow other kernels too
    for rep in range(n_reps):
        for N in Ns:
            t0 = time.time()
            try:
                res = run_example_scenario(
                    N=N, d=2, K=25,
                    cov_type=kernel,
                    cov_params=cov_params or {},
                    R=20,
                    widths=[64,64,64],
                    lr=1e-2,
                    max_epochs=800,
                    patience=80,
                    M_mc=M_mc,
                    verbose=False
                )
                rel_l2 = res['rel_l2']
                fit_time = res['fit_time']
            except Exception as e:
                rel_l2 = None; fit_time = None
                print(f"[Scenario B] error kernel={kernel} N={N} rep={rep}: {e}")
            results.append({
                'scenario': 'B_convergence',
                'kernel': kernel,
                'R': 20,
                'N': N, 'd': 2, 'K': 25,
                'rep': rep,
                'rel_l2': rel_l2,
                'fit_time': fit_time,
                'timestamp': time.time()
            })

# Scenario C: Dimensional scaling — vary d and K (N=50)
# d_values = [2, 3]
# K_values = [5, 10]
# for kernel, cov_params in kernel_configs.items():
#     for rotate in ([False, True] if kernel in rotatable else [False]):
#         for rep in range(n_reps):
#             for d in d_values:
#                 for K in K_values:
#                     t0 = time.time()
#                     try:
#                         res = run_example_scenario(
#                             N=50, d=d, K=K,
#                             cov_type=kernel, cov_params=cov_params or {},
#                             R=10, widths=[128,128,128],
#                             lr=1e-2, max_epochs=800, patience=80, M_mc=M_mc, verbose=False
#                         )
#                         rel_l2 = res['rel_l2']; fit_time = res['fit_time']
#                     except Exception as e:
#                         rel_l2 = None; fit_time = None
#                         print(f"[Scenario C] error kernel={kernel} rotate={rotate} d={d} K={K} rep={rep}: {e}")
#                     results.append({
#                         'scenario': 'C_dim_scaling',
#                         'kernel': kernel,
#                         'rotated': rotate,
#                         'R': 10,
#                         'N': 50, 'd': d, 'K': K,
#                         'rep': rep,
#                         'rel_l2': rel_l2,
#                         'fit_time': fit_time,
#                         'timestamp': time.time()
#                     })

# # Scenario D: Noise & discretization — use generate_dataset directly to add noise
# noise_levels = [0.1, 0.5, 1.0, 2.0]
# K_values = [10, 25]
# for kernel, cov_params in kernel_configs.items():
#     for rep in range(n_reps):
#         for K in K_values:
#             for sigma2_noise in noise_levels:
#                 d = 3
#                 N = 500
#                 pts = grid_on_unit_cube(d, K)
#                 t0 = time.time()
#                 try:
#                     X = generate_dataset(N, pts, cov_type=kernel, cov_params=cov_params or {},
#                                          jitter=1e-8, sigma2_noise=sigma2_noise, rotate=False)
#                     model, Xi, hist = train_deepshared(X, pts, R=20, widths=[64,64,64],
#                                                        lr=1e-2, max_epochs=800, patience=80, verbose=False)
#                     c_true_fn = true_cov_function_factory(pts, cov_type=kernel, cov_params=cov_params or {}, rotate=False)
#                     c_est_fn = make_c_est_from_deepshared(model, Xi, pts)
#                     rel_l2 = monte_carlo_rel_L2(c_true_fn, c_est_fn, d=d, M=M_mc, seed=123 + rep)
#                     fit_time = time.time() - t0
#                 except Exception as e:
#                     rel_l2 = None; fit_time = None
#                     print(f"[Scenario D] error kernel={kernel} K={K} noise={sigma2_noise} rep={rep}: {e}")
#                 results.append({
#                     'scenario': 'D_noise_disc',
#                     'kernel': kernel,
#                     'R': 20,
#                     'N': N, 'd': d, 'K': K,
#                     'rep': rep,
#                     'sigma2_noise': sigma2_noise,
#                     'rel_l2': rel_l2,
#                     'fit_time': fit_time,
#                     'timestamp': time.time()
#                 })

# -----------------------
# Save summary
# -----------------------
df = pd.DataFrame(results)
csv_path = OUTDIR / "deepshared_sim_summary.csv"
df.to_csv(csv_path, index=False)
print(f"Saved summary to {csv_path}")
print(df.groupby(['scenario','kernel'])['rel_l2'].agg(['count','mean','std']).reset_index())


C:\Users\Carter\miniconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Carter\miniconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Grid points D: 625
Fitting finished in 18.942957878112793 sec
Relative L2 error (MC): 0.7933626958781224
Grid points D: 625
Fitting finished in 20.559967517852783 sec
Relative L2 error (MC): 0.8126924208009083
Grid points D: 625
Fitting finished in 17.254873991012573 sec
Relative L2 error (MC): 0.8139836038680992
Grid points D: 625
Fitting finished in 17.608361959457397 sec
Relative L2 error (MC): 0.7835311232188059
Grid points D: 625
Fitting finished in 18.523252248764038 sec
Relative L2 error (MC): 0.8143930600231015
Grid points D: 625
Fitting finished in 17.522228717803955 sec
Relative L2 error (MC): 0.8340318401712885
Grid points D: 625
Fitting finished in 18.41474962234497 sec
Relative L2 error (MC): 0.8001749194544051
Grid points D: 625
Fitting finished in 17.516191482543945 sec
Relative L2 error (MC): 0.8372870244798492
Grid points D: 625
Fitting finished in 17.30406165122986 sec
Relative L2 error (MC): 0.816279857697875
Grid points D: 625
Fitting finished in 17.604840993881226 

## Notes
- The notebook focuses on the DeepShared CovNet estimator and provides a runnable example.  
- Edit hyperparameters / experiment runner cells to reproduce Scenario A-D described in the simulation plan (only DeepShared is implemented).